<a href="https://colab.research.google.com/github/tqthanhcong/streamlit-example/blob/master/Get_bank_saving_interest_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import package

In [ ]:
!pip install -q tabula-py #extracting table from pdf
!pip install img2pdf
!sudo apt install tesseract-ocr #OCR
!pip install pytesseract #OCR
!pip install kora #selenium + chromedriver.exe for google colab

     |████████████████████████████████| 11.7MB 256kB/s 
     |████████████████████████████████| 51kB 3.7MB/s 
     |████████████████████████████████| 1.9MB 13.9MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 ht

In [ ]:
#import essential package

#dataframe
import numpy as np
import pandas as pd
import json

#scraping
import requests
from kora.selenium import wd
from bs4 import BeautifulSoup
import urllib.request

#pdf handling
import tabula #table from pdf
import img2pdf #image to pdf

#OCR
import cv2 
import pytesseract

#time out
from datetime import datetime
from dateutil import tz
import time
import signal #timeout
import re

from contextlib import contextmanager

# Create function

In [ ]:
#function for setting timeout
@contextmanager
def timeout(time):
    # Register a function to raise a TimeoutError on the signal.
    signal.signal(signal.SIGALRM, raise_timeout)
    # Schedule the signal to be sent after ``time``.
    signal.alarm(time)

    try:
        yield
    except TimeoutError:
        pass
    finally:
        # Unregister the signal so it won't be triggered
        # if the timeout is not reached.
        signal.signal(signal.SIGALRM, signal.SIG_IGN)


def raise_timeout(signum, frame):
    raise TimeoutError

In [ ]:
#Create function that extract table data from html
def get_table(bank,url,attr,table_index,row_start,row_end,col_term,col_interest):
  with timeout(10):
    r = requests.get(url,verify=False)
    html_doc = r.text
    soup=BeautifulSoup(html_doc,'lxml')
    table = soup.find_all('table',attrs=attr)[table_index]
    df = pd.DataFrame(columns=range(0,1000), index = range(500))
    row_marker = 0
    for row in table.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            df.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        row_marker += 1
    df = df.iloc[row_start:row_end,[col_term,col_interest]]
    df.columns = ['term','interest']
    df['bank'] = bank
    return df

#function for converting html table to dataframe
def html_to_df(table,df):
  row_marker = 0
  for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
      df.iat[row_marker,column_marker] = column.get_text()
      column_marker += 1
    row_marker += 1
  return df

#Create function that extract table data from a json link
def get_json(url):
    r = requests.get(url)
    html_doc = r.text
    #chuyển dữ liệu trả về thành dataframe để xử lý và lọc
    html_list = json.loads(html_doc)
    return html_list

#function for downloading pdf file
def get_pdf_from_url(bank_name,url):
  with timeout(20):
    #r = requests.get(url)
    opener = urllib.request.URLopener()
    opener.addheader('User-Agent', 'whatever')
    posi=url.rfind('.')
    extension = url[posi:]
    if (len(extension)>5)|(len(extension)<2):
      extension='.pdf'
    file_open, headers = opener.retrieve(url, bank_name + extension)
    #file_open = open(bank_name + extension,'wb').write(r.content)
   

#function for extracting table from pdf file
def extract_table(name,table_index,row_start,row_end,col_term,col_interest):
  tables = tabula.read_pdf(name+'.pdf',pages='all',multiple_tables=True)
  df = tables[table_index]
  df=df.iloc[row_start:row_end,[col_term,col_interest]]
  df.columns = ['term','interest']
  df['bank'] = name
  return df


In [ ]:
#Create the original dataframe
df_all = pd.DataFrame(columns = ['term','interest','bank'])

#create error list
err_list=[]

# Get interest from HTML Table

In [ ]:
#list of link for each bank
html_url = {'1. AGR':'https://www.agribank.com.vn/vn/lai-suat',
            '3. CTG':'https://www.vietinbank.vn/web/home/vn/lai-suat',
            '4. VCB':'https://portal.vietcombank.com.vn/UserControls/TVPortal.TyGia/pListLaiSuat.aspx?CusstomType=1&BacrhID=1&InrateType=&isEn=False&numAfter=2',
            '11. LPB':'https://lienvietpostbank.com.vn/lai-suat',
            '14. ABB':'https://www.abbank.vn/thong-tin/lai-suat-tiet-kiem-vnd.html',
            '15. OCB':'https://www.ocb.com.vn/vi/lai-suat.html',
            '16. SCB':'https://www.scb.com.vn/vie/detaillist/qa_interestrate_detail/QA_INTERESTRATE_NORMAL_SAVING_DEPOSIT/0?_=1599729793501',
            '21. MSB':'https://www.msb.com.vn/Bieu-mau-Bieu-phi-va-Lai-suat/1177/3/Lai-suat',
            '23. EAB':'https://kinhdoanh.dongabank.com.vn/widget/temp/-/DTSCDongaBankIView_WAR_DTSCDongaBankIERateportlet?type=tktt-vnd',
            '25. NAB':'https://www.namabank.com.vn/lai-suat-tien-gui-vnd-nam',
            '28. BVB':'https://www.vietcapitalbank.com.vn/personal',
            '29. KLB':'https://laisuat.kienlongbank.com/lai-suat-ca-nhan',
            '34. OCEAN':'http://oceanbank.vn/lai-suat.html',
            '35. GPBANK':'https://www.gpbank.com.vn/InterestDetail'
             }
#html_index: table_index,row_start,row_end,col_term,col_interest
html_index = {'1. AGR':[{},0,1,16,0,1],
              '3. CTG':[{},0,3,21,0,1],
              '4. VCB':[{},0,2,13,0,1],
              '11. LPB':[{},1,0,26,0,4],
              '14. ABB':[{},0,1,22,0,1],
              '15. OCB':[{},0,1,13,0,1],
              '16. SCB':[{},0,3,19,0,1],
              '21. MSB':[{},2,2,19,0,1],
              '23. EAB':[{},1,1,20,0,3],
              '25. NAB':[{},0,1,39,0,1],
              '28. BVB':[{},3,0,19,0,3],
              '29. KLB':[{},1,3,24,0,1],
              '34. OCEAN':[{'class':'tb_lstk tble_2'},0,3,24,0,1],
              '35. GPBANK':[{},0,2,22,0,6]
              }

In [ ]:
for bank in html_url.keys():
  try:
    #bank,url,attr,table_index,row_start,row_end,col_term,col_interest
    df=get_table(bank,html_url[bank],html_index[bank][0],html_index[bank][1],\
                 html_index[bank][2],html_index[bank][3],html_index[bank][4],html_index[bank][5])
    df_all = df_all.append(df).reset_index(drop=True)
  except:
    err_list.append(bank)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

# Get interest from JSON

In [ ]:
#BIDV - JSON
#send request to server and recieve responsed text
try:
  url = 'https://www.bidv.com.vn/ServicesBIDV/InterestDetailServlet'
  html_list = get_json(url)

  html_list = html_list['hcm']['data']
  df = pd.DataFrame(html_list)[['title_vi','VND']]
  df.columns=['term','interest']
  df['bank']='2. BID'
  df_all = df_all.append(df).reset_index(drop=True)
except:
  err_list.append('2. BID')

In [ ]:
#VIB - JSON: the data need to get is "TDE" product (pay at the end)
#send request to server and recieve responsed text
#error when clean if concat to df_VIB_all now, so I do it later at the clean data cell
try:
  url_json = 'https://www.vib.com.vn/VIBOpenCA/html/jsp/info/new_getSavingRateFromCore.jsp'
  html_list = get_json(url_json)
  df_VIB=pd.DataFrame(html_list[0],index=[0,])
  for i in range(1,len(html_list)):
    dict = html_list[i]
    df_VIB = df_VIB.append(pd.DataFrame(dict,index=[0,]))

  df_VIB=df_VIB.loc[df_VIB['productType']=='TDE',['term','actualRate','effectDate','balance']]
  df_VIB=df_VIB[df_VIB['balance']=='4999999999']
  df_VIB=df_VIB.loc[df_VIB['effectDate']==df_VIB['effectDate'].max(),['term','actualRate','effectDate','balance']]
  df_VIB = df_VIB.drop_duplicates(subset = 'term')
  df_VIB = df_VIB.iloc[:,[0,1]]
  df_VIB.columns=['term','interest']
  df_VIB['bank']='12. VIB'
except:
  err_list.append('12. VIB')


In [ ]:
#BaoViet Bank
try:
  url='https://baovietbank.vn/AdTools/GetListInterestByCurr?currency=VND&groupCode=59956903-780b-43a1-970d-cec9ec77cc0f'
  wd.get(url)
  time.sleep(5)
  body=wd.find_element_by_tag_name('body')
  text = body.text
  my_list = json.loads(text)
  df=pd.DataFrame(my_list)
  df.drop('id',axis=1,inplace=True)
  df=df[df['producT_CODE_TERM']=='6601'] #Interest received at the end of the period
  df=df[['terM_INTEREST','interesT_INTEREST_DATE']]
  df.columns=['term','interest']
  df['bank']='26. BV'
  df_all = df_all.append(df).reset_index(drop=True)
except:
  err_list.append('26. BV')

In [ ]:
#PGBank
try:
  url = "https://www.pgbank.com.vn/api/v1/get-interest-rate?lang=vi"
  r= requests.get(url)
  dict = r.json()
  dict = dict["data"]
  df=pd.DataFrame(dict)
  df["term"]=df["Tenor"].astype(str)+df["TenorType"].astype(str)
  df = df.iloc[:,[-1,-2]]
  df.columns=['term','interest']
  df['bank']='33. PGB'
  df_all = df_all.append(df).reset_index(drop=True)
except:
  err_list.append('33. PGB')

# Get table from page source by selenium

Put all code in 1 cell will raise error, maybe the second line code run when the first line code (get) doesn't finish

In [ ]:
#MBBank
try:
  wd.get('https://mbbank.com.vn/fee')
  time.sleep(5)
  source = wd.page_source
  soup = BeautifulSoup(source)
  table = soup.find_all('div',attrs={'id':'rate-info0'})[0]
  df = pd.DataFrame(columns=range(0,1000), index = range(500))
  df=html_to_df(table,df)
  df= df.iloc[1:24,[0,1]]
  df.columns=['term','interest']
  df['bank']='5. MBB'
  df_all = df_all.append(df).reset_index(drop=True)
except:
  err_list.append('5. MBB')

In [ ]:
#PVCOMBANK
try:
  url= 'https://www.pvcombank.com.vn/bieu-lai-suat.html'
  wd.get(url)
  time.sleep(5)
  form_control=wd.find_element_by_xpath('//*[@id="slSavingProduct"]')
  form_control.click()
  li = form_control.find_elements_by_tag_name('option')
  for i in range(len(li)):
    if li[i].text=='[GD Tại quầy] Tiết kiệm Đại chúng trả lãi cuối kỳ':
      li[i].click()
  time.sleep(1)
  html_doc=wd.page_source
  soup = BeautifulSoup(html_doc)
  table=soup.find('table',attrs={'id':'tableInterestRate'})
  df = pd.DataFrame(columns=range(0,1000), index = range(500))
  df=html_to_df(table,df)
  df=df.iloc[2:22,[0,1]]
  df.columns=['term','interest']
  df['bank']='19. PV'
  df_all = df_all.append(df).reset_index(drop=True)
except:
  err_list.append('19. PV')

In [ ]:
#SEABANK
try:
  url= 'https://www.seabank.com.vn/site#/interest'
  wd.get(url)
  time.sleep(5)
  form_control=wd.find_element_by_xpath('//*[@id="main"]/div[1]/div[3]/div/div/div[2]/select')
  form_control.click()
  li = form_control.find_elements_by_tag_name('option')
  for i in range(len(li)):
    if li[i].text=='TIET KIEM LINH LAI CUOI KY':
      li[i].click()
  html_doc=wd.page_source
  soup = BeautifulSoup(html_doc)
  table=soup.find('table',attrs={'class':'table table-striped'})
  df = pd.DataFrame(columns=range(0,1000), index = range(500))
  df=html_to_df(table,df)
  df=df.iloc[1:20,[2,3]]
  df.columns=['term','interest']
  df['bank']='20. SEA'
  df_all = df_all.append(df).reset_index(drop=True)
except:
  err_list.append('20. SEA')

In [ ]:
#SEABANK
try:
  url= 'https://www.seabank.com.vn/site#/interest'
  wd.get(url)
  time.sleep(10)
  form_control=wd.find_element_by_xpath('//*[@id="main"]/div[1]/div[3]/div/div/div[2]/select')
  form_control.click()
  li = form_control.find_elements_by_tag_name('option')
  for i in range(len(li)):
    if li[i].text=='TIET KIEM LINH LAI CUOI KY':
      li[i].click()
  html_doc=wd.page_source
  soup = BeautifulSoup(html_doc)
  table=soup.find('table',attrs={'class':'table table-striped'})
  df = pd.DataFrame(columns=range(0,1000), index = range(500))
  df=html_to_df(table,df)
  df=df.iloc[1:20,[2,3]]
  df.columns=['term','interest']
  df['bank']='20. SEA'
  df_all = df_all.append(df).reset_index(drop=True)
except:
  err_list.append('20. SEA')

In [ ]:
#Bac A Bank
try:
  url= 'https://www.baca-bank.vn/SitePages/website/lai-xuat.aspx?ac=L%C3%A3i%20su%E1%BA%A5t&s=LX'
  wd.get(url)
  time.sleep(5)
  xp = '//*[@id="ctl00_PlaceHolderMain_g_3d987530_2758_4587_b052_bda8fbe88390_ctl00_DDL_LS"]'
  form_control=wd.find_element_by_xpath(xp)
  form_control.click()
  li = form_control.find_elements_by_tag_name('option')
  for i in range(len(li)):
    if li[i].text=='VNĐ':
      li[i].click()
  html_doc=wd.page_source
  soup = BeautifulSoup(html_doc)
  table=soup.find_all('table')[0]
  df = pd.DataFrame(columns=range(0,1000), index = range(500))
  df=html_to_df(table,df)
  df=df.iloc[3:23,[0,3]]
  df.columns=['term','interest']
  df['bank']='22. BAB'
  df_all = df_all.append(df).reset_index(drop=True)
except:
  err_list.append('22. BAB')

In [ ]:
#SGB
try:
  wd.get('https://www.saigonbank.com.vn/vi/truy-cap-nhanh/lai-suat')
  time.sleep(5)
  source = wd.page_source
  soup = BeautifulSoup(source)
  table = soup.find_all('table')[0]
  df = pd.DataFrame(columns=range(0,1000), index = range(500))
  df=html_to_df(table,df)
  df= df.iloc[4:22,[0,1]]
  df.columns=['term','interest']
  df['bank']='30. SGB'
  df_all = df_all.append(df).reset_index(drop=True) 
except:
  err_list.append('30. SGB')

In [ ]:
#NCB
try:
  wd.get('https://www.ncb-bank.vn/vi/lai-suat-tien-gui')
  time.sleep(5)
  source = wd.page_source
  soup = BeautifulSoup(source)
  table = soup.find_all('table',attrs={'class':'tb-content'})[0]
  df = pd.DataFrame(columns=range(0,1000), index = range(500))
  df=html_to_df(table,df)
  df= df.iloc[4:25,[0,1]]
  df.columns=['term','interest']
  df['bank']='31. NCB'
  df_all = df_all.append(df).reset_index(drop=True)
except:
  err_list.append('31. NCB')

In [ ]:
#CBBank
try:
  wd.get('https://www.cbbank.vn/Pages/InterestRate.aspx')
  time.sleep(10)
  source = wd.page_source
  soup = BeautifulSoup(source)
  table = soup.find_all('table',attrs={'class':'MsoNormalTable'})[0]
  df = pd.DataFrame(columns=range(0,1000), index = range(500))
  df=html_to_df(table,df)
  df= df.iloc[4:26,[0,1,2]]
  df[0]=df[0].str.replace('\n','') + ' '
  df[1]=df[1].str.replace('\n','')
  df[2]=df[2].str.replace('\n','')
  df[3]=df[0]+''+df[1]
  df=df[[3,2]]
  df.columns=['term','interest']
  df['bank']='32. CBBANK'
  df_all = df_all.append(df).reset_index(drop=True)
except:
  err_list.append('32. CBBANK')

# Get link for downloading pdf

In [ ]:
#find url for downloading file

#Sacombank
try:
  with timeout(5):
    url = 'https://www.sacombank.com.vn/canhan/Pages/lai-suat.aspx'
    r=requests.get(url)
    soup=BeautifulSoup(r.text,'lxml')
    interates=soup.find_all('div',attrs={'class':'fee-item mrb-40'})[0]
    deposit = interates.find_all('a')[0]
    url = deposit['href']
    url_stb ='https://www.sacombank.com.vn'+url
except:
  pass

try:
  #VietABank
  url = 'https://www.vietabank.com.vn/tien-ich/lai-suat-tien-gui/lai-suat-khach-hang-ca-nhan.html'
  r=requests.get(url)
  soup=BeautifulSoup(r.text,'lxml')
  interates=soup.find_all('div',attrs={'class':'interest-group'})
  interate = interates[0]
  url_container = interate.find_all('img')[0]
  url = url_container['src']
  url_vab='https://www.vietabank.com.vn'+url
except:
  pass

try:
  #VietBank
  url = 'https://www.vietbank.com.vn/ca-nhan/ho-tro/lai-suat'
  r=requests.get(url,verify=False)
  soup=BeautifulSoup(r.text,'lxml')
  interates=soup.find_all('div',attrs={'class':'item-bieumau'})
  interate = interates[3]
  url_container = interate.find_all('a')[0]
  url_vbb = url_container['href']
except:
  pass

#Techcombank
try:
  with timeout(15):
    url = 'https://www.techcombank.com.vn/cong-cu-tien-ich/lai-suat'
    r=requests.get(url,verify=False)
    soup=BeautifulSoup(r.text,'lxml')
    interates=soup.find_all('li',attrs={'class':'item-rates item-rates-select'})
    interate = interates[1]
    url_container = interate.find_all('a')[0]
    url_tcb = 'https://www.techcombank.com.vn' + url_container['href']
except:
  url_tcb=''
  pass

#TPBank
url = 'https://tpb.vn/wps/wcm/connect/tpbank_data/sa-website/ty-gia-lai-suat-cong-cu-tinh-toan/lai-xuat/danh-cho-khach-hang-ca-nhan/danh-cho-khach-hang-ca-nhan?source=library&srv=cmpnt&cmpntid=c3723702-cc5e-43bc-8123-867223571d0a'
r=requests.get(url)
soup=BeautifulSoup(r.text,'lxml')
interates=soup.find_all('p',attrs={'dir':'ltr'})
interate = interates[1]
url_container = interate.find_all('a')[0]
url_tpb = 'https://tpb.vn' + url_container['href']

#VPBank
url = 'https://www.vpbank.com.vn/tai-lieu-bieu-mau'
r=requests.get(url)
soup=BeautifulSoup(r.text,'lxml')
interates=soup.find_all('div',attrs={'id':'category_3'})
interate = interates[0]
url_container = interate.find('div',attrs={'class':'nav-tab__content__detail'})
url = url_container.find('a')
url_vpb = 'https://www.vpbank.com.vn' + url['href']

#Eximbank
url = 'https://www.eximbank.com.vn/khachhangcanhan'
r=requests.get(url)
soup=BeautifulSoup(r.text,'lxml')
interates=soup.find_all('div',attrs={'class':'collapse show'})[0]
interates2=interates.find_all('div',attrs={'class':'card-body'})[0]
interates3=interates2.find_all('div',attrs={'class':'col-md-4 col-lg-6'})[0]
interates4=interates3.find_all('ul')
interate = interates4[2]
url = interate.find('a')
url_eib = 'https://www.eximbank.com.vn' + url['href']

#ACB
try:
  url = 'https://www.acb.com.vn/wps/portal?1dmy&page=acb.viewer&urile=wcm%3Apath%3AACBWebsite/acb-vn/personal/tai-khoan-tien-gui/khac/lai-suat'
  r=requests.get(url)
  soup=BeautifulSoup(r.text,'lxml')
  url=soup.find('a',attrs={'class':'breadcrumb'})
  url_acb = 'https://www.acb.com.vn' + url['href']
except:
  url_acb=''
#SHB
url = 'https://www.shb.com.vn/category/lien-ket-nhanh/lai-suat/'
r=requests.get(url)
soup=BeautifulSoup(r.text,'lxml')
interates=soup.find_all('div',attrs={'class':'detail'})[0]
url_container=interates.find_all('a')[0]
url_shb = url_container['href']
url_shb = url_shb.replace('\\/\\/','//')

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
try:
  #VPBank
  url = 'https://www.vpbank.com.vn/tai-lieu-bieu-mau'
  r=requests.get(url)
  soup=BeautifulSoup(r.text,'lxml')
  interates=soup.find_all('div',attrs={'id':'category_3'})
  interate = interates[0]
  url_container = interate.find('div',attrs={'class':'nav-tab__content__detail'})
  url = url_container.find('a')
  url_vpb = 'https://www.vpbank.com.vn' + url['href']
except:
  pass

In [ ]:
#Download all pdf file
urls = {
        '6. ACB':url_acb,
        '7. SHB':url_shb,
        '8. TCB':url_tcb,
        '9. VPB':url_vpb,
        #'10. HDB':url_hdb,
        '13. TPB':url_tpb,
        '17. STB':url_stb,
        #'24. VAB':url_vab,
        '27. VBB':url_vbb,
        '18. EIB':url_eib
        }                 
for bank in urls.keys():
  try:
    get_pdf_from_url(bank,urls[bank])
  except:
    try:
      headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0"}
      r=requests.get(urls[bank],headers=headers)
      with open(bank + '.pdf','wb') as f:
        f.write(r.content)
    except:
      pass
time.sleep(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  app.launch_new_instance()


# Extract table from pdf

In [ ]:
# table index, begin row, ending row, term column, interate column
banks = {
                '6. ACB':[1,5,-1,0,2],
                '8. TCB':[0,5,-1,0,2],
                '10. HDB':[0,5,28,0,1],
                '13. TPB':[0,9,-1,0,5],
                '17. STB':[0,4,-1,0,1],
                '27. VBB':[0,2,-1,0,1],
                '18. EIB':[0,2,-1,0,6]
                }

for bank in banks.keys():
  try:
    #name,table_index,row_start,row_end,col_term,col_interest
    df=extract_table(bank,banks[bank][0],\
                     banks[bank][1],banks[bank][2],banks[bank][3],banks[bank][4])
    df_all = df_all.append(df).reset_index(drop=True)
  except:
    err_list.append(bank)

Got stderr: May 27, 2021 2:04:14 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
May 27, 2021 2:04:15 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
May 27, 2021 2:04:15 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>

Got stderr: May 27, 2021 2:04:22 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 27, 2021 2:04:22 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 27, 2021 2:04:26 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 27, 2021 2:04:27 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 27, 2021 2:04:27 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 27, 2021 2:04:45 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 27, 2021 2:04:45 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 27, 2021 2:04:45 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 27, 2021 2:04:46 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 27,

Some table must be transposed

In [ ]:
#VPB
try:
  tables = tabula.read_pdf('9. VPB.pdf',pages='all',multiple_tables=True)
  df = tables[0].transpose().iloc[3:,[1,6]].reset_index(drop=True)
  df.columns = ['term','interest']
  df['bank'] = '9. VPB'
  df['term']=df['term'].str.replace('T','M')
  df_all = df_all.append(df).reset_index(drop=True)
except:
  err_list.append('9. VPB')

In [ ]:
#SHB
#bi loi do phan lai suat trong file pdf la hinh anh
try:
  tables = tabula.read_pdf('7. SHB.pdf',pages='all',multiple_tables=True)
  df = tables[2].transpose().iloc[1:,[0,2]].reset_index(drop=True)
  df.columns = ['term','interest']
  df['bank'] = '7. SHB'
  df['term']=df['term'].str.replace('T','M').str.replace('>=','')
  df_all = df_all.append(df).reset_index(drop=True)
except:
  err_list.append('7. SHB')

# OCR

In [ ]:
try:
  #Viet A Bank
  #extract text form jpg file
  img = cv2.imread('24. VAB.jpg')
  custom_config = r'--oem 3 --psm 6'
  data=pytesseract.image_to_data(img,lang='eng',config=custom_config,output_type='data.frame')
  data=data.dropna(subset=['text'])

  #create some list
  text = data['text'].values
  conf=data['conf'].values
  par=data['par_num'].values
  line=data['line_num'].values
  left=data['left'].values
  text_list=[[]]
  left_list=[[]]

  #convert text to list of list of words, based on their line
  row=0
  for i in range(len(par)):
    if i==0:
      pass
    elif (par[i]!=par[i-1])|(line[i]!=line[i-1]):
      row+=1
      text_list.append([])
      left_list.append([])
    text_list[row].append(text[i])
    left_list[row].append(left[i])

  #find the position of interate table
  for i in range(len(text_list)):
    if (text_list[i][0]=='Ky')&(text_list[i][1]=='han'):
      break
  start_line=i+1
  for i in range(start_line,len(text_list)):
    if (text_list[i][0]=='36'):
      break
  end_line=i
  table=text_list[start_line:end_line+1]

  #extract infomation from the table and put to dataframe
  term=[]
  interate=[]
  for i in range(len(table)):
    term.append(table[i][0]+'M')
    interate.append(table[i][2])
  df=pd.DataFrame(zip(term,interate))

  #clean and add to df_all
  df.columns = ['term','interest']
  df['bank'] = '24. VAB'
  df_all = df_all.append(df).reset_index(drop=True)
except:
  pass

# Clean final data

In [ ]:
df_backup = df_all.copy()
#df_all=df_backup.copy()

In [ ]:

#ACB
subset = (df_all['bank']=='6. ACB') & (df_all['term']!= '13 Tháng') & (df_all['term'] != '1 – 3 tuần')
df_all.loc[subset,'interest'] = df_all.loc[subset,'interest'].str[10:14]
#TCB
subset = (df_all['bank']=='8. TCB') & (df_all['term']!='12M')
df_all.loc[subset,'interest'] = df_all.loc[subset,'interest'].str[:4].str.strip()
#TPB
try:
  subset1 = (df_all['bank']=='13. TPB') & (df_all['term']=='12M')
  subset2 = (df_all['bank']=='13. TPB') & (df_all['term']=='364 ngày')
  df_all.loc[df_all['bank']=='13. TPB','interest']=df_all.loc[df_all['bank']=='13. TPB','interest'].str[0:4].str.strip()
  df_all.loc[subset1,'interest']=df_all.loc[subset2,'interest'].values[0]
except:
  pass
df_all.shape

(637, 3)

In [ ]:
term_replace = {'\xa0':' ',
                '\r':'',
                '\n':'',
                'Không kỳ hạn':'KKH',
                'không kỳ hạn':'KKH',
                ' tuần':'W',
                ' Tuần':'W',
                ' tháng':'M',
                'tháng':'M',
                ' Tháng':'M',
                ' ngày':'D',
                ' Ngày':'D',
                'Dưới 1 tháng':'1W',
                '7D':'1W',
                '14D':'2W',
                '21D':'3W'}
#replace ký tự
for term in df_all['term']:
  for old_char in term_replace.keys():
    df_all['term'] = df_all['term'].str.replace(old_char,term_replace[old_char])

#lấy số bỏ chữ
for i in range(len(df_all['term'])):
  try:
    string = str(df_all['term'][i])
    position = re.search(r'\d', string).start()
    df_all['term'][i] = string[position:position+4].replace(' ','')
    #bỏ số 0 ở đầu
    if df_all['term'][i][0]=='0':
      df_all['term'][i]=df_all['term'][i][1:]
  except: pass

#replace ký tự lãi suất
interest_replace = {',':'.',
                    '%':'',
                    '\xa0':' ',
                    '\r':'',
                    '\n':'',
                    '(i)':'',
                    '-':''
                    }

for interest in df_all['interest']:
  for old_char in interest_replace.keys():
    df_all['interest'] = df_all['interest'].str.replace(old_char,interest_replace[old_char]).str.strip()

df_all['interest'] = df_all['interest'].str[0:4].str.strip()
df_all = df_all.dropna(how='all')
df_all = df_all.append(df_VIB).reset_index(drop=True)

In [ ]:
term_list = {'term':['KKH','1W','1M','2M','3M','6M','9M','12M','13M','15M','18M','24M','36M','60M'],
             'term_index':range(14)}
df_term_list = pd.DataFrame(term_list)
df_term_list['key']=1
bank = df_all['bank'].drop_duplicates()
df_bank = pd.DataFrame({'bank':bank}).reset_index(drop=True)
df_bank['bank_index']=df_bank['bank'].str[0:2].str.replace('.','').astype('float')
df_bank['key']=1
bank_term = pd.merge(df_term_list,df_bank,on='key').drop('key',1)
df_all_merge = df_all.merge(bank_term,on=['bank','term'],how='right')
df_all_merge = df_all_merge.sort_values(['bank_index','term_index']).reset_index(drop=True)
df_all_merge['date']=datetime.today().astimezone(tz.gettz('Asia/Hanoi'))

In [ ]:
#fill na
for i in range(len(df_all_merge['term'])):
  if pd.isnull(df_all_merge.iloc[i,1]):
    if (df_all_merge.iloc[i,0] == 'KKH'):
      df_all_merge.iloc[i,1] = 0.1
    else:
      df_all_merge.iloc[i,1] = df_all_merge.iloc[i-1,1]

In [ ]:
df_all_merge.drop_duplicates(subset=['term','bank'])
df_all_merge.to_csv('interest.csv')

# Pivot table

In [ ]:
df_add = df_all_merge.head(0)
df_add['bank_index']=range(1,36)
df_add['bank']=['1. AGR', '2. BID', '3. CTG', '4. VCB', '5. MBB', '6. ACB', '7. SHB', '8. TCB', '9. VPB', '10. HDB', '11. LPB', '12. VIB', '13. TPB', '14. ABB', '15. OCB', '16. SCB', '17. STB', '18. EIB', '19. PV', '20. SEA', '21. MSB', '22. BAB', '23. EAB', '24. VAB', '25. NAB', '26. BV', '27. VBB', '28. BVB', '29. KLB', '30. SGB', '31. NCB', '32. CBBANK', '33. PGB', '34. OCEAN', '35. GPBANK']
df_add.loc[:,'term']='KKH'
df_add.loc[:,'interest']=0
df_add.loc[:,'term_index']=0
df=df_all_merge.append(df_add)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [ ]:
df.loc[df['interest']=='','interest']=np.nan
#df['interest']=df['interest'].str.replace("0 .2","0.2")
df['interest']=df['interest'].astype('float')
pvt =df.pivot_table(values='interest',index=['bank_index','bank']\
                         ,columns=['term_index','term'],dropna=True,aggfunc='max')
pvt.to_excel('interest.xlsx')
print(pvt)
df_backup.to_csv('before_cleaning.csv')

term_index               0     1     2     3   ...    10    11    12    13
term                    KKH    1W    1M    2M  ...   18M   24M   36M   60M
bank_index bank                                ...                        
1.0        1. AGR      0.10  0.10  3.10  3.10  ...  5.60  5.60  5.60  5.60
2.0        2. BID      0.10  0.10  3.10  3.10  ...  5.60  5.60  5.60  5.60
3.0        3. CTG      0.10  0.10  3.10  3.10  ...  5.60  5.60  5.60  5.60
4.0        4. VCB      0.10  0.20  2.90  2.90  ...  5.50  5.30  5.30  5.30
5.0        5. MBB      0.10  0.10  3.00  3.10  ...  5.70  5.35  6.20  6.20
6.0        6. ACB      0.10  0.10  3.15  3.25  ...  6.20  6.20  6.20  6.20
7.0        7. SHB      0.00   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN
8.0        8. TCB      0.03  0.03  2.45  2.45  ...  4.50  4.50  4.50  4.50
9.0        9. VPB      0.10  0.10  3.38  3.70  ...  5.30  5.30  5.30  5.30
10.0       10. HDB     0.00   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN
11.0       11. LPB     0.

In [ ]:
print(err_list)

['31. NCB', '10. HDB', '7. SHB']
